In [1]:
import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode
import torch


In [2]:
path = "../informatie/apple_disease_classification/images/Train/Dataset/normal_apples/good_apple0.jpg"
img = read_image(path, ImageReadMode.UNCHANGED)/255

print(img.shape)
print(img)

torch.Size([3, 128, 128])
tensor([[[0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         [0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         [0.9059, 0.9059, 0.9059,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.9490, 0.9490, 0.9490,  ..., 0.9725, 0.9725, 0.9725],
         [0.9529, 0.9529, 0.9529,  ..., 0.9725, 0.9725, 0.9725],
         [0.9529, 0.9529, 0.9529,  ..., 0.9725, 0.9725, 0.9725]],

        [[0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         [0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         [0.9216, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.9412, 0.9412, 0.9412,  ..., 0.9529, 0.9529, 0.9529],
         [0.9451, 0.9451, 0.9451,  ..., 0.9529, 0.9529, 0.9529],
         [0.9451, 0.9451, 0.9451,  ..., 0.9529, 0.9529, 0.9529]],

        [[0.9569, 0.9569, 0.9569,  ..., 0.9686, 0.9686, 0.9686],
         [0.9569, 0.9569, 0.9569,  ..., 0.9686, 0.9686, 0.9686],
         [0.9569, 0.9569, 0.9569

In [3]:

dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"

In [4]:
class DatasetAppels(Dataset):
    def __init__(self, img_folder_path, transform):

        image_folder = ImageFolder(img_folder_path, transform=transform)
        len_classes = len(image_folder.class_to_idx)
        self.images = [image[0] for image in image_folder]
        self.labels = [torch.Tensor(self.labeler(image[1], len_classes)) for image in image_folder]
        self.class_dict = image_folder.class_to_idx

    def labeler(self, label, len_classes):
        result = [0] * len_classes
        result[label] = 1
        return result

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return [self.images[idx], self.labels[idx]]
    


In [5]:
dataset = DatasetAppels(dataset_path, T.ToTensor())

dataset

In [6]:
print(dataset.labels)
print(dataset.class_dict)

[tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1., 0., 0., 0.]), tensor([1.,

In [7]:
print(len(dataset))

2845


In [8]:
generator1 = torch.Generator().manual_seed(13)

# create a train test split with 70% train, 30% test
train_dataset, test_dataset, val_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=generator1)

# check length of train and test dataset
print(len(train_dataset), len(test_dataset), len(val_dataset))

# create train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

1707 569 569


In [9]:
class CNN(nn.Module):
    
    def __init__(self, img_size=128, c_in=3, loss_func=nn.CrossEntropyLoss()):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(c_in, 16, 3, padding=1),
            nn.ReLU(),
            nn. Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn. Conv2d(64, 96, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),

            nn.Linear(img_size*768, 500),
            nn.ReLU(),
            nn.Linear(500, 256),
            nn.ReLU(),
            nn.Linear(256, 4),
            nn.Softmax(dim=1)
            )
        
        self.loss_func = loss_func

    def fit(self, train_loader, val_loader, test_loader, epochs, lr, opt_function=torch.optim.Adam):
        optimizer = opt_function(self.model.parameters(),lr )
        history = []
        self.model.to(self.cuda_available())

        for epoch in range(epochs):
            print(epoch)
            self.model.train()
            

            for batch in train_loader:
                optimizer.zero_grad()
                loss = self.loss_calc(batch)
                loss.backward()
                optimizer.step()
                
            with torch.no_grad():    
                self.model.eval()

                val_loss = []
                for batch in val_loader:
                    loss = self.loss_calc(batch)
                    val_loss.append(loss)
                
            
            history.append(sum(val_loss))
            print(sum(val_loss))
        
            print("Accuracy:",self.evaluate_accuracy(test_loader))
        return history
    
    def cuda_available(self):

        if torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")
                
    def loss_calc(self, batch):
        image, labels = batch
        
        image = image.to(self.cuda_available())
        labels = labels.to(self.cuda_available())
        pred = self(image)
        loss = self.loss_func(pred, labels)
        return loss


    def evaluate_accuracy(self, test_loader):         
        cor_pred = 0
        bad_pred = 0
        for batch in test_loader:
            image, labels = batch
            image = image.to(self.cuda_available())
            labels = labels.to(self.cuda_available())
            pred = self(image)
    
            _, y_pred = torch.max(pred,1)                       
            
            for i in range(len(y_pred)):
                result = [0] * len(pred[0])

                p = int(y_pred[i])
                result[p] = 1
                if result == labels[i].tolist():
                    cor_pred += 1
                else:
                    bad_pred += 1

        acc = cor_pred/(cor_pred + bad_pred) * 100
        return acc     
    
    def predict(self, image):
        pred = self(image)
        _, y_pred = torch.max(pred,1)
        print(f"y_pred: {y_pred}")                       
        
        for i in range(len(y_pred)):
            result = [0] * len(pred[0])
            print(f"len(pred[0]): {len(pred[0])}")

            p = int(y_pred[i])
            print(f"p: {p}")
            result[p] = 1

        return result

    

        


    
    def forward(self, data):

        return self.model(data)


In [10]:
# Loop over dataloader and push images and labels to device
# dataset.images.to("cuda")
# dataset.labels.to("cuda")

net = CNN()

net.fit(train_loader, val_loader, test_loader, lr = 0.0001, epochs=10)
net.fit(train_loader, val_loader, test_loader, lr = 0.00005, epochs=5)


0
tensor(127.1308, device='cuda:0')
Accuracy: 62.56590509666081
1
tensor(117.3444, device='cuda:0')
Accuracy: 68.7170474516696
2
tensor(118.3241, device='cuda:0')
Accuracy: 68.89279437609842
3
tensor(113.6345, device='cuda:0')
Accuracy: 73.4622144112478
4
tensor(114.2160, device='cuda:0')
Accuracy: 73.98945518453426
5
tensor(106.8457, device='cuda:0')
Accuracy: 79.26186291739896
6
tensor(108.2210, device='cuda:0')
Accuracy: 79.43760984182776
7
tensor(107.5204, device='cuda:0')
Accuracy: 77.85588752196837
8
tensor(105.9886, device='cuda:0')
Accuracy: 80.31634446397187
9
tensor(106.8716, device='cuda:0')
Accuracy: 80.14059753954305
0
tensor(104.0441, device='cuda:0')
Accuracy: 84.18277680140598
1
tensor(106.3933, device='cuda:0')
Accuracy: 81.89806678383128
2
tensor(104.2154, device='cuda:0')
Accuracy: 83.83128295254832
3
tensor(103.2879, device='cuda:0')
Accuracy: 84.71001757469244
4
tensor(101.9819, device='cuda:0')
Accuracy: 87.17047451669596


[tensor(104.0441, device='cuda:0'),
 tensor(106.3933, device='cuda:0'),
 tensor(104.2154, device='cuda:0'),
 tensor(103.2879, device='cuda:0'),
 tensor(101.9819, device='cuda:0')]

In [14]:
test_path = "Project 3/informatie/apple_disease_classification/images/Test/Normal_Apple/34.jpg"
test_img = read_image(path, ImageReadMode.UNCHANGED)/255

resize = T.Resize((128,128))
resize(test_img)
test_img = test_img.unsqueeze(0)
test_img.shape

torch.Size([1, 3, 128, 128])

In [17]:
test_img = test_img.to(torch.device("cuda"))
net.predict(test_img)

y_pred: tensor([2], device='cuda:0')
len(pred[0]): 4
p: 2


[0, 0, 1, 0]

In [13]:
# def evaluate_accuracy(self, test_loader):         
#         cor_pred = 0
#         bad_pred = 0
#         for batch in test_loader:
#             image, labels = batch
#             image = image.to(self.cuda_available())
#             labels = labels.to(self.cuda_available())
#             result = self.predict(image)

#             for i in range(len(result)):

#                 if result == labels[i].tolist():
#                     cor_pred += 1
#                 else:
#                     bad_pred += 1

#         acc = cor_pred/(cor_pred + bad_pred) * 100
#         return acc     
    
#     def predict(self, image):
#         pred = self(image)
#         _, y_pred = torch.max(pred,1)                       
        
#         for i in range(len(y_pred)):
#             result = [0] * len(pred[0])

#             p = int(y_pred[i])
#             result[p] = 1

#         return result